In [2]:
import numpy as np
import pandas as pd
import mne
import pathlib

from scipy.stats import pearsonr
from scipy.stats import wilcoxon

from tensorflow.keras.models import load_model

from mne.preprocessing import ICA

In [3]:
# Define the DataFrame with appropriate columns
columns = [
    "Segment_ID",
    "RMS_Noisy",
    "RMS_Cleaned",
    "RMS_diff",
    "Correlation",
    "P_Value",
]
results_df = pd.DataFrame(columns=columns)

In [4]:
data = {}
for filename in pathlib.Path("./lee2019-artifacts/").glob("*Artifact*.fif"):
    data[filename] = mne.io.read_raw(filename, preload=True, verbose=False).pick(
        ["O1", "O2", "Fp1", "Fp2"]
    )

segment_length = 2
all_segments = []

for filename, raw in data.items():
    n_segments = int(np.floor(raw.times[-1] / segment_length))

    for i in range(n_segments):
        start_sec = i * segment_length
        stop_sec = start_sec + segment_length
        stop_sec = min(stop_sec, raw.times[-1])

        cropped_raw = raw.copy().crop(
            tmin=start_sec, tmax=stop_sec - 1 / raw.info["sfreq"]
        )
        all_segments.append(cropped_raw)

In [5]:
def rms(raw):
    """
    Computes the Root Mean Square of the signal, providing a measure of the overall power in the signal.
    """
    data = raw.get_data()
    rms = np.sqrt(np.mean(data**2))
    return np.round(rms, 4)

In [6]:
def correlation(noisy, clean):
    """
    Calculates the Pearson correlation coefficient for each channel between the
    noisy and cleaned signals to assess how well the overall structure of the signal
    is preserved post-cleaning.
    """
    correlation_coeffs = []
    for ch in range(4):
        coeff, _ = pearsonr(noisy.get_data(picks=ch)[0], clean.get_data(picks=ch)[0])
        correlation_coeffs.append(coeff)

    average_corr = np.mean(correlation_coeffs)

    return np.round(average_corr, 4)

In [7]:
def significance(noisy, clean):
    """
    Uses the Wilcoxon signed-rank test to determine if there is a
    statistically significant difference between the noisy and cleaned EEG signals.
    """
    data_original = noisy.get_data().flatten()
    data_cleaned = clean.get_data().flatten()

    _, p = wilcoxon(data_original, data_cleaned)

    return p

In [8]:
model = load_model("./RNN_model/models/b20-LRsch.keras")

In [9]:
def bilstm_pipeline(raw, model):
    raw.filter(1, 40, fir_design="firwin")
    raw.resample(256)

    processed_raw = raw.copy()
    data = processed_raw.get_data()

    # Process input data
    std_devs = np.std(data, axis=1, keepdims=True)
    data_standardized = data / std_devs

    t = 2
    samples_per_segment = t * int(processed_raw.info["sfreq"])
    n_channels = data_standardized.shape[0]

    segments = []
    for channel in range(n_channels):
        channel_data = data_standardized[channel, :]
        n_segments = channel_data.shape[0] // samples_per_segment
        for segment in range(n_segments):
            start = segment * samples_per_segment
            end = start + samples_per_segment
            if end <= channel_data.shape[0]:
                segments.append(
                    channel_data[start:end].reshape(samples_per_segment, 1)
                )
    segments = np.array(segments)

    # model = load_model("./RNN_model/models/b20-LRsch.keras")

    # Denoise data
    denoised_data = []
    for segment in segments:
        prediction = model.predict(
            segment[np.newaxis, :, :]
        )  # Model predicts on one segment at a time
        denoised_data.append(prediction.squeeze())
    denoised_data = np.array(denoised_data)

    # Initialize the denoised array with the correct shape
    denoised_full = np.zeros(data.shape)
    for channel in range(n_channels):
        channel_std_dev = std_devs[channel]
        n_segments = data[channel, :].shape[0] // samples_per_segment
        for segment in range(n_segments):
            start = segment * samples_per_segment
            end = start + samples_per_segment
            if end <= data[channel, :].shape[0]:
                denoised_full[channel, start:end] = (
                    denoised_data[segment * n_channels + channel] * channel_std_dev
                )

    info = mne.create_info(
        ch_names=processed_raw.info["ch_names"],
        sfreq=processed_raw.info["sfreq"],
        ch_types="eeg",
    )

    # Reconstruct signal
    reconstructed = mne.io.RawArray(denoised_full, info)

    return reconstructed

In [10]:
for i, noisy_segment in enumerate(all_segments):
    cleaned = bilstm_pipeline(noisy_segment, model)

    rms_noisy = rms(noisy_segment)
    rms_cleaned = rms(cleaned)
    corr = correlation(noisy_segment, cleaned)
    try:
        p_value = significance(noisy_segment, cleaned)
    except:
        p_value = None

    results_df = results_df.append(
        {
            "Segment_ID": i,
            "RMS_Noisy": rms_noisy,
            "RMS_Cleaned": rms_cleaned,
            "RMS_diff": np.abs(rms_cleaned - rms_noisy),
            "Correlation": corr,
            "P_Value": p_value,
        },
        ignore_index=True,
    )

results_df.to_csv("bilstm_analysis_results.csv", index=False)

1/1 [==============================] - 0s 82ms/step


C:\Users\rasa\AppData\Local\Temp\ipykernel_17112\72381304.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(


1/1 [==============================] - 0s 83ms/step


C:\Users\rasa\AppData\Local\Temp\ipykernel_17112\72381304.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(


1/1 [==============================] - 0s 82ms/step


...

C:\Users\rasa\AppData\Local\Temp\ipykernel_17112\72381304.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(


1/1 [==============================] - 0s 83ms/step


C:\Users\rasa\AppData\Local\Temp\ipykernel_17112\72381304.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(


1/1 [==============================] - 0s 87ms/step


C:\Users\rasa\AppData\Local\Temp\ipykernel_17112\72381304.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(


In [11]:
# Basic descriptive statistics
print(results_df.describe())

# Filter segments with significant improvements
significant_improvements = results_df[results_df["P_Value"] < 0.05]
print("Significant Improvements Count:", significant_improvements.shape[0])

        Segment_ID    RMS_Noisy  RMS_Cleaned     RMS_diff  Correlation  \
count  3093.000000  3093.000000  3093.000000  3093.000000  3093.000000   
mean   1546.000000    69.402310     3.214740    66.187570     0.027524   
std     893.016517    51.475036     2.218786    49.350477     0.180769   
min       0.000000     5.348600     0.345300     4.993100    -0.674400   
25%     773.000000    35.930400     1.747500    34.129200    -0.087100   
50%    1546.000000    58.704300     2.710500    56.023100     0.038800   
75%    2319.000000    86.596100     4.059900    82.620500     0.146500   
max    3092.000000   495.824300    22.214500   476.569500     0.618300   

             P_Value  
count   3.093000e+03  
mean    6.369242e-02  
std     1.840099e-01  
min    3.770559e-175  
25%     5.171899e-28  
50%     9.304075e-11  
75%     2.209275e-03  
max     9.966016e-01  
Significant Improvements Count: 2600
